In [1]:
import ktrain
import numpy as np
import pandas as pd
from IPython.display import display, HTML
from ktrain import text

using Keras version: 2.2.4-tf


In [2]:
DATA_PATH = r'train.csv'
NUM_WORDS = 50000
MAXLEN = 264

In [3]:
(x_train, y_train), (x_test, y_test), preproc = text.texts_from_csv(DATA_PATH,
                      'Text',
                      label_columns = ['STRENGTHS','WEAKNESSES','OPPORTUNITIES','THREATS'],
                      preprocess_mode='bert',
                      val_filepath=None, 
                      max_features=NUM_WORDS,
                      maxlen=MAXLEN)

detected encoding: ISO-8859-3 (if wrong, set manually)
preprocessing train...
language: en


preprocessing test...
language: en


In [4]:
x_test[:5]

[array([[  101,  1996,  2194, ...,     0,     0,     0],
        [  101, 17473,  8089, ...,     0,     0,     0],
        [  101,  3811,  6975, ...,     0,     0,     0],
        ...,
        [  101,  2045,  2038, ...,     0,     0,     0],
        [  101,  5100,  1999, ...,     0,     0,     0],
        [  101,  2429,  2000, ...,     0,     0,     0]]),
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])]

In [5]:
y_test[:5]

array([[0, 0, 1, 0],
       [0, 1, 0, 0],
       [0, 0, 0, 1],
       [0, 1, 0, 0],
       [0, 0, 1, 0]], dtype=int64)

In [6]:
model = text.text_classifier('bert', train_data=(x_train, y_train), preproc=preproc)

Is Multi-Label? False
maxlen is 264
done.


In [7]:
learner = ktrain.get_learner(model, train_data=(x_train, y_train), batch_size=16)

In [8]:
learner.fit_onecycle(2e-5, 4)



begin training using onecycle policy with max lr of 2e-05...
Train on 2250 samples
Epoch 1/4
2250/2250 [==============================] - 1829s 813ms/sample - loss: 1.0817 - accuracy: 0.5413
Epoch 2/4
2250/2250 [==============================] - 1811s 805ms/sample - loss: 0.4932 - accuracy: 0.8231
Epoch 3/4
2250/2250 [==============================] - 1765s 785ms/sample - loss: 0.2972 - accuracy: 0.8911
Epoch 4/4
2250/2250 [==============================] - 1765s 785ms/sample - loss: 0.1565 - accuracy: 0.9502


In [9]:
learner.validate(val_data=(x_test, y_test), class_names=['S','W','O','T'])

              precision    recall  f1-score   support

           S       0.92      0.92      0.92        87
           W       0.83      0.82      0.83        55
           O       0.95      0.91      0.93        58
           T       0.89      0.94      0.91        50

    accuracy                           0.90       250
   macro avg       0.90      0.90      0.90       250
weighted avg       0.90      0.90      0.90       250



array([[80,  6,  1,  0],
       [ 4, 45,  1,  5],
       [ 3,  1, 53,  1],
       [ 0,  2,  1, 47]], dtype=int64)

In [10]:
predictor = ktrain.get_predictor(learner.model, preproc)

In [11]:
predictor.get_classes()

['STRENGTHS', 'WEAKNESSES', 'OPPORTUNITIES', 'THREATS']

In [12]:
df1 = pd.read_excel(r"test.xlsm",index=1)

In [13]:
df1.set_index("#id", inplace = True)

In [14]:
display(HTML(df1[15:16].to_html()))

,Text,Label,STRENGTHS,WEAKNESSES,OPPORTUNITIES,THREATS
#id,,,,,,
2516,"As informed by management, it also incurs substantial cost in foreign currency which to some extent mitigates the risk. Forward cover of 40 - 75% on each invoice value is booked depending up on volatility visible in the market generally for one full month and with 90 days maturity. Currently, the Group is booking forward cover for 50% on all invoices (USD, GBP, and AUD) value to shield against foreign currency movement. In FY 2019, Group has booked a forex loss of around INR 4.8 crores (forming 1.69% of total revenue).",4,0,0,0,1


In [15]:
predictor.predict(np.array(df1['Text'][15:16]))

['WEAKNESSES']

In [17]:
# let's save the predictor for later use
predictor.save(r'.\run264_16\saved_model_25k')

In [18]:
# reload the predictor
reloaded_predictor = ktrain.load_predictor(r'.\run264_16\saved_model_25k')

In [19]:
# make a prediction on the same document to verify it still works
results=reloaded_predictor.predict(np.array(df1['Text'][0:16]))

In [20]:
df1['Prediction']=results
display(HTML(df1.to_html()))

,Text,Label,STRENGTHS,WEAKNESSES,OPPORTUNITIES,THREATS,Prediction
#id,,,,,,,
2501,"As informed, the Group has competitive edge over peers due to its robust customer support service, ease in using software modules and value-added functionalities like analytics built-in the software which to some extent ensures the client stickiness along with long established relations with the client with about 90% of business coming from repeat orders.",2,0,1,0,0,WEAKNESSES
2502,"The group primarily caters to the international market with the domestic market contributing around 1% of the total revenue during FY 2018. In the overseas market its geographical base is spread across United States of America (USA), United Kingdom, Europe, Middle East and Asia Pacific regions; however, clients from USA contributed about 70% to the total revenue. Thus the Group is exposed to risk associated with geographically concentrated operations. Any slowdown in the US market, could have a direct impact on the business of the Group.",2,0,1,0,0,WEAKNESSES
2503,"As informed by the management, it primarily caters to billion dollar plus revenue companies and thus finds the US as an appropriate market for its products. Additionally, it is increasingly exploring customers across Asia Pacific, Middle East as well as the domestic market. Consequently, geographic concentration to US is expected to reduce going forward. The group has forayed into multiple new countries including Netherland, Singapore and Dubai through wholly owned subsidiaries.",2,0,1,0,0,WEAKNESSES
2504,Collection period of the Group remained high and ranged between 113 and 128 days during the five years ended 31st March 2019. It is significantly high when compared with the management’s credit policy of 30-60 days. The increase in collection period can lengthen the working capital cycle which in turn can impact the liquidity of the Group.,2,0,1,0,0,WEAKNESSES
2505,"As informed, the stretched collection period was largely due to high invoicing towards the end of the year (Q4) resulting in accumulation of receivables. About 30-40% of the invoicing is done in the last month of the year ie March. Average credit days allowed to its customers otherwise stood at 30-60 days. The Group is currently able to manage the high receivables through own internal accruals, However, stretched working capital cycle could result in incremental fund requirement which if funded through external borrowings may leverage the capital structure and lead to interest expenses impacting the profitability of the Company.",2,0,1,0,0,WEAKNESSES
2506,Increasing thrust on improving efficiencies to drive growth of the Group,3,0,0,1,0,OPPORTUNITIES
2507,"Chief procurement officers require insightful procurement and spend management intelligence to cope with increasing complexities in the global market. Procurement software helps client accelerate and automate procurement function. With layer of analytics built into the solution, the client gets complete real time visibility of purchase trends in the organization alongside multi-dimensional reporting. Streamlining purchase processes will help officers to centralize and have a tighter control of all purchase transactions. Hence, with continuous need to improve efficiency in the competitive environment, demand for procurement solution software such as that of Zycus will continue to increase further.",3,0,0,1,0,OPPORTUNITIES
2508,"The Group is aggressively expanding in other regions through its newly incorporated subsidiaries and has increased its targeted focus in Middle East, South-East Asia, and certain European countries for new market expansion. Also, it has added 30 new marketing personnel. With expanded reach, the contribution from USA reduced from 90% in FY 2017 to around 70% in FY 2019. The Group aims to further reduce exposure to USA to 50% with balance 50% expected to be derived from other regions, mainly Europe. Additional sales force and penetrating in newer regions is expected to r